In [5]:
pip install --upgrade gradio


Note: you may need to restart the kernel to use updated packages.


In [50]:
import gradio as gr
import joblib
import re

# Load the trained model and vectorizer
model = joblib.load("random_forest_model.pkl")  # Ensure correct path
vectorizer = joblib.load("vectorizer3.pkl")  # Load vectorizer

# Preprocessing function
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Prediction function with confidence score
def predict_review(review):
    processed_review = clean_text(review)  # Preprocess input text
    transformed_review = vectorizer.transform([processed_review])  # Convert to vector
    
    prediction = model.predict(transformed_review)[0]  # Get prediction (0 or 1)
    confidence = model.predict_proba(transformed_review)[0]  # Get probability scores
    
    fake_confidence = confidence[1]  # Probability of being a fake review
    genuine_confidence = confidence[0]  # Probability of being a genuine review

    result = "human generated" if prediction == 1 else "computer generated"
    confidence_score = max(fake_confidence, genuine_confidence)  # Get highest probability

    return f"{result} (Confidence: {confidence_score:.2%})"  # Format as percentage

# Gradio Interface
iface = gr.Interface(
    fn=predict_review,
    inputs=gr.Textbox(label="Enter Review"),
    outputs=gr.Label(label="Prediction"),
    title="Fake Review Detector with Confidence Score",
    description="paste a review to check if it's fake or genuine.",
)

iface.launch(share = True)

* Running on local URL:  http://127.0.0.1:7872
* Running on public URL: https://81d5ab36a4aab46441.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
